In [1]:
#pip install git+https://github.com/JustAnotherArchivist/snscrape

In [1]:
import pandas as pd
import snscrape.modules.twitter
import gc
import time
import datetime

### 1) Ticker tweets

In [2]:
since = '2012-01-01'
until = '2022-05-01'
query = '(gme) min_faves:10 since:{0} until:{1}'.format(since,until)
#query = '(aapl OR msft OR googl OR amzn OR fb OR tsla OR brk.a OR tsm OR nvda OR jpm) min_faves:10 since:{0} until:{1}'.format(since,until)
scraper = snscrape.modules.twitter.TwitterSearchScraper(query)
i=0
tweets_update = pd.DataFrame()
gen = scraper.get_items()
while True:
    i+=1
    try: 
        tweet = next(gen)
        tweets_update = pd.concat([tweets_update, pd.DataFrame([{"url": tweet.url,
                                        "date": tweet.date,
                                        "content": tweet.content,
                                        "id": tweet.id,
                                        "username": tweet.user.username,
                                        "replyCount": tweet.replyCount,
                                        "retweetCount": tweet.retweetCount,
                                        "likeCount": tweet.likeCount,
                                        "lang": tweet.lang,
                                        "hashtags": tweet.hashtags,
                                        "cashtags": tweet.cashtags}])])
        
        
    except StopIteration:
        tweets = pd.read_csv("data/tweets_tickers.csv")
        tweets = pd.concat([tweets, tweets_update])
        tweets.to_csv("data/tweets_tickers.csv",index=False)
        del tweets
        gc.collect()
        break
    except:
        print('error')
        


    
    if i%10000 == 0:
        print('First date in year: ' + str(pd.to_datetime(tweets_update.date,utc=True).min()))
        tweets = pd.read_csv("data/tweets_tickers.csv")
        tweets = pd.concat([tweets, tweets_update])
        tweets.to_csv("data/tweets_tickers.csv",index=False)
        del tweets
        gc.collect()
        tweets_update = pd.DataFrame()



First date in year: 2022-03-30 14:45:59+00:00
First date in year: 2022-03-04 09:14:13+00:00
First date in year: 2022-01-24 21:38:34+00:00
First date in year: 2021-12-17 14:53:56+00:00
First date in year: 2021-11-10 21:40:56+00:00
First date in year: 2021-10-07 17:22:10+00:00
First date in year: 2021-08-31 14:06:57+00:00
First date in year: 2021-07-20 15:25:45+00:00
First date in year: 2021-06-11 19:11:21+00:00
First date in year: 2021-05-14 14:09:57+00:00
First date in year: 2021-03-24 02:38:09+00:00
First date in year: 2021-02-11 18:12:27+00:00
First date in year: 2021-01-30 00:26:48+00:00
First date in year: 2021-01-28 15:40:44+00:00
First date in year: 2021-01-27 10:24:57+00:00
First date in year: 2020-04-16 20:05:18+00:00
First date in year: 2018-07-23 01:00:21+00:00
First date in year: 2015-08-18 12:03:32+00:00


### 2) Keyword tweets

In [2]:
keywords = {
    "AAPL" : "(aapl OR apple OR \"steve jobs\" OR \"steve wozniak\" OR \"ronald wayne\" OR \"tim cook\" OR macintosh OR mac OR ios OR ipod OR iphone OR ipad OR airpods OR homepod OR icloud OR itunes)",
    "MSFT" : "(msft OR microsoft OR \"bill gates\" OR \"paul allen\" OR windows OR office OR skype OR \"visual studio\" OR xbox OR azure OR bing OR linkedin OR yammer OR onedrive OR outlook OR github OR \"game pass\" OR sharepoint OR \"visual studio\" OR azure)",
    "GOOGL" : "(googl OR alphabet OR google OR \"larry page\" OR \"sergey brin\" OR \"sundar pichai\" OR gmail OR firebase OR tensorflow OR android OR chrome OR adwords OR nexus OR pixel OR youtube OR deepmind)",
    "AMZN" : "(amzn OR amazon OR \"jeff bezos\" OR \"andy jassy\" OR kindle OR alexa OR \"fire tv\" OR \"fire tablet\" OR aws OR audible OR goodreads OR imdb OR twitch)",
    "FB" : "(fb OR meta OR \"mark zuckerberg\" OR zuckerberg OR \"eduardo saverin\" OR metaverse OR facebook OR messenger OR instagram OR whatsapp OR oculus OR mapillary)",
    "TSLA" : "(tsla OR tesla OR \"elon musk\" OR elon OR musk OR \"model s\" OR cybertruck OR \"model x\" OR \"model y\" OR \"model 3\" OR powerwall OR powerpack OR deepscale OR solarcity)",
    "BRK-A" : "(brk.a OR brk.b OR brk OR \"berkshire hathaway\" OR \"oliver chace\" OR \"warren buffett\" OR \"buffett\" OR \"altalink\" OR \"kern river pipeline\" OR \"northern natural gas\" OR \"fruit of the loom\" OR \"netjets\" OR \"russell brands\")",
    "TSM" : "(2330 OR tsm OR tsmc OR \"taiwan semiconductor\" OR \"morris chang\" OR \"mark liu\" OR cybershuttle OR wafertech OR ssmc)",
    "NVDA" : "(nvda OR nvidia OR \"jensen huang\" OR geforce OR mellanox OR physx OR deepmap)",
    "JPM" : "(jpm OR jpmorgan OR \"j.p. morgan\" OR \"john pierpont morgan\" OR \"jamie dimon\" OR \"chase bank\" OR \"one equity partners\")",
    "GME" : "(gme OR gamestop OR \"ryan cohen\" OR \"matt furlong\")"
    }

In [9]:
since = '2012-01-01'
until = '2022-05-01'

since_unix =  str(int(time.mktime(datetime.datetime.strptime(since, "%Y-%m-%d").timetuple())))
until_unix =  str(int(time.mktime(datetime.datetime.strptime(until, "%Y-%m-%d").timetuple())))


ticker = "GOOGL" # SELECT TICKER!
query = keywords[ticker] +' min_faves:100 since:{0} until:{1}'.format(since_unix,until_unix)
last_query = query
scraper = snscrape.modules.twitter.TwitterSearchScraper(query)
i = 0
j = 0
stuck = False

tweets_update = pd.DataFrame()
gen = scraper.get_items()
while True:
    i+=1
    try: 
        tweet = next(gen)
        tweets_update = pd.concat([tweets_update, pd.DataFrame([{"url": tweet.url,
                                        "date": tweet.date,
                                        "content": tweet.content,
                                        "id": tweet.id,
                                        "username": tweet.user.username,
                                        "replyCount": tweet.replyCount,
                                        "retweetCount": tweet.retweetCount,
                                        "likeCount": tweet.likeCount,
                                        "lang": tweet.lang,
                                        "hashtags": tweet.hashtags,
                                        "cashtags": tweet.cashtags,
                                        "ticker": ticker}])])
        if not stuck:
            j = 0

        
        
        
    except StopIteration:
        tweets = pd.read_csv("data/tweets_keywords.csv")
        tweets = pd.concat([tweets, tweets_update])
        tweets.to_csv("data/tweets_keywords.csv",index=False)
        del tweets
        gc.collect()
        print('finished')
        break
        
    except:

        
        print("updating-query, j: ",j )
        last_query = query
        until_unix = str(int(time.mktime(datetime.datetime.strptime(str(tweets_update["date"].min()), "%Y-%m-%d %H:%M:%S%z").timetuple()))+7000-j*3600)
        query = keywords[ticker] +' min_faves:100 since:{0} until:{1}'.format(since_unix,until_unix)
        scraper = snscrape.modules.twitter.TwitterSearchScraper(query)
        gen = scraper.get_items()
        j+=1
        
        if last_query==query:
            stuck = True
        else:
            stuck = False
        

        


    
    if i%10000 == 0:
        print(ticker + ' --- First date in year: ' + str(pd.to_datetime(tweets_update.date,utc=True).min()))
        tweets = pd.read_csv("data/tweets_keywords.csv")
        tweets = pd.concat([tweets, tweets_update])
        tweets.to_csv("data/tweets_keywords.csv",index=False)
        del tweets
        gc.collect()
        tweets_update = pd.DataFrame()



GOOGL --- First date in year: 2020-09-06 04:33:24+00:00
GOOGL --- First date in year: 2020-08-31 17:26:31+00:00
GOOGL --- First date in year: 2020-08-27 01:00:07+00:00
GOOGL --- First date in year: 2020-08-22 06:23:14+00:00
GOOGL --- First date in year: 2020-08-17 21:57:57+00:00
GOOGL --- First date in year: 2020-08-12 11:19:52+00:00
GOOGL --- First date in year: 2020-08-06 21:08:59+00:00
GOOGL --- First date in year: 2020-08-01 08:34:25+00:00
GOOGL --- First date in year: 2020-07-27 02:30:21+00:00
GOOGL --- First date in year: 2020-07-21 16:37:51+00:00
GOOGL --- First date in year: 2020-07-15 23:44:32+00:00
GOOGL --- First date in year: 2020-07-10 09:53:29+00:00
GOOGL --- First date in year: 2020-07-04 19:46:18+00:00
GOOGL --- First date in year: 2020-06-29 21:27:04+00:00
GOOGL --- First date in year: 2020-06-25 15:58:54+00:00
GOOGL --- First date in year: 2020-06-20 13:44:18+00:00
GOOGL --- First date in year: 2020-06-15 18:14:03+00:00
GOOGL --- First date in year: 2020-06-10 10:49:0

C:\Users\Pracovní\AppData\Local\Temp\ipykernel_13276\2335487841.py:72: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets = pd.read_csv("data/tweets_keywords.csv")


GOOGL --- First date in year: 2020-04-09 10:41:23+00:00
GOOGL --- First date in year: 2020-04-03 11:48:24+00:00
GOOGL --- First date in year: 2020-03-28 11:45:55+00:00
GOOGL --- First date in year: 2020-03-22 07:12:47+00:00
GOOGL --- First date in year: 2020-03-16 01:52:18+00:00
GOOGL --- First date in year: 2020-03-09 08:00:00+00:00
GOOGL --- First date in year: 2020-03-02 07:35:51+00:00
GOOGL --- First date in year: 2020-02-24 14:09:14+00:00
GOOGL --- First date in year: 2020-02-16 18:45:31+00:00
GOOGL --- First date in year: 2020-02-08 13:16:42+00:00
GOOGL --- First date in year: 2020-01-31 16:18:48+00:00
GOOGL --- First date in year: 2020-01-23 09:30:00+00:00
GOOGL --- First date in year: 2020-01-14 22:30:02+00:00
GOOGL --- First date in year: 2020-01-06 22:11:08+00:00
GOOGL --- First date in year: 2019-12-29 00:21:41+00:00
GOOGL --- First date in year: 2019-12-20 00:38:55+00:00
GOOGL --- First date in year: 2019-12-11 14:29:08+00:00
GOOGL --- First date in year: 2019-12-03 14:18:1

Could not translate t.co card URL on tweet 966947686221049856


GOOGL --- First date in year: 2018-02-11 03:13:30+00:00
GOOGL --- First date in year: 2018-01-24 00:05:55+00:00
GOOGL --- First date in year: 2018-01-07 10:36:22+00:00
GOOGL --- First date in year: 2017-12-21 06:44:03+00:00
GOOGL --- First date in year: 2017-12-03 02:44:55+00:00
GOOGL --- First date in year: 2017-11-13 00:03:37+00:00
GOOGL --- First date in year: 2017-10-20 14:05:21+00:00


...

GOOGL --- First date in year: 2015-12-25 12:28:32+00:00


C:\Users\Pracovní\AppData\Local\Temp\ipykernel_13276\2335487841.py:72: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  tweets = pd.read_csv("data/tweets_keywords.csv")
Could not translate t.co card URL on tweet 667333141174992896
Could not translate t.co card URL on tweet 666796903250591745
Could not translate t.co card URL on tweet 666589836451561472


GOOGL --- First date in year: 2015-10-29 09:49:03+00:00


Could not translate t.co card URL on tweet 659014362166759424
Could not translate t.co card URL on tweet 657906211245821952
Could not translate t.co card URL on tweet 648971723900108800


GOOGL --- First date in year: 2015-08-26 17:52:43+00:00
GOOGL --- First date in year: 2015-06-17 20:26:06+00:00
GOOGL --- First date in year: 2015-04-05 15:31:26+00:00
GOOGL --- First date in year: 2015-01-18 01:27:24+00:00
GOOGL --- First date in year: 2014-10-15 18:47:05+00:00
GOOGL --- First date in year: 2014-06-29 05:18:18+00:00
GOOGL --- First date in year: 2014-01-30 07:36:51+00:00
GOOGL --- First date in year: 2013-01-26 00:26:36+00:00
finished
